<a href="https://colab.research.google.com/github/fahad3345/Charcter-level-seq2seq-model-for-English-French-translation/blob/main/English_to_French_Translator_by_words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

## Download the data

In [2]:
!!curl -O http://www.manythings.org/anki/fra-eng.zip
!!unzip fra-eng.zip

['Archive:  fra-eng.zip',
 '  inflating: _about.txt              ',
 '  inflating: fra.txt                 ']

## Configuration

In [3]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "fra.txt"

## Prepare the data

In [5]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
lines[:10]

['Go.\tVa !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)',
 'Go.\tMarche.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8090732 (Micsmithel)',
 'Go.\tEn route !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8267435 (felix63)',
 'Go.\tBouge !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #9022935 (Micsmithel)',
 'Hi.\tSalut !\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #509819 (Aiji)',
 'Hi.\tSalut.\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #4320462 (gillux)',
 'Run!\tCours\u202f!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #906331 (sacredceltic)',
 'Run!\tCourez\u202f!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #906332 (sacredceltic)',
 'Run!\tPrenez vos jambes à vos cous !\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #2077449 (sacredceltic)',
 'Run!\tFile !\tCC-BY 2.0 (France) Attribution: tatoeba.org #90

In [9]:
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
print('Input unique characters:',input_characters)
print('Target unique characters:',target_characters)

Input unique characters: {'&', '-', 'A', 'z', 'o', 'V', 'q', ' ', 'O', 'r', 'i', 'j', 'G', 'H', '9', ':', 'c', '0', 't', "'", 'U', 'u', '5', 'e', '.', 'Q', 'F', 'L', '!', '%', '?', 'f', 'h', 'B', '"', 'N', '$', 'K', 'w', 'b', 'm', '3', 'S', '7', 'W', 'k', 'R', '1', 'T', '8', '2', 'n', 'd', 'x', 'C', 's', 'M', 'P', 'y', 'I', 'Y', 'a', 'g', 'l', 'v', ',', 'p', 'J', 'E', 'D'}
Target unique characters: {'-', 'V', '\u202f', 'r', '9', ')', 'ê', '»', '\xa0', '\n', 'ù', 'K', '1', 'û', 'C', 'é', 's', 'E', 'î', 'A', 'ç', 'z', '’', 'i', 'Ç', 'Ê', "'", 'u', '\u2009', 'h', 'B', 'N', 'b', '2', 'R', 'Y', 'g', 'l', ',', 'J', '\t', 'q', ' ', 'O', 'H', '0', 'c', 'à', '5', 'e', '?', 'è', 'â', 'œ', '3', 'w', 'S', 'W', '8', 'ï', 'd', 'M', 'x', 'P', '«', 'É', '(', 'À', '&', 'o', 'j', 'G', ':', 't', 'U', '.', 'Q', 'F', 'L', '!', '%', 'f', 'm', 'T', 'k', 'ô', 'n', 'y', 'I', 'a', 'v', 'p', 'D'}


In [13]:
#input_texts
#target_texts

In [14]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [15]:
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 40000
Number of unique input tokens: 70
Number of unique output tokens: 93
Max sequence length for inputs: 14
Max sequence length for outputs: 59


In [16]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [19]:
#input_token_index
#target_token_index

In [20]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

In [27]:
print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)

(40000, 14, 70)
(40000, 59, 93)
(40000, 59, 93)


In [34]:
#This code doing the One Hot Encoding part
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [38]:
#encoder_input_data

## Model building

In [39]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

In [40]:
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [43]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

In [44]:
# We set up our decoder to return full output sequences,and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

In [47]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

## Model training

In [48]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s")

Epoch 1/100
500/500 [==============================] - 16s 16ms/step - loss: 0.9413 - accuracy: 0.7578 - val_loss: 0.7324 - val_accuracy: 0.7906
Epoch 2/100
500/500 [==============================] - 6s 13ms/step - loss: 0.6268 - accuracy: 0.8177 - val_loss: 0.5996 - val_accuracy: 0.8239
Epoch 3/100
500/500 [==============================] - 6s 13ms/step - loss: 0.5415 - accuracy: 0.8412 - val_loss: 0.5325 - val_accuracy: 0.8440
Epoch 4/100
500/500 [==============================] - 7s 13ms/step - loss: 0.4866 - accuracy: 0.8569 - val_loss: 0.4838 - val_accuracy: 0.8568
Epoch 5/100
500/500 [==============================] - 6s 11ms/step - loss: 0.4462 - accuracy: 0.8682 - val_loss: 0.4501 - val_accuracy: 0.8670
Epoch 6/100
500/500 [==============================] - 7s 14ms/step - loss: 0.4158 - accuracy: 0.8768 - val_loss: 0.4252 - val_accuracy: 0.8738
Epoch 7/100
500/500 [==============================] - 6s 12ms/step - loss: 0.3907 - accuracy: 0.8839 - val_loss: 0.4007 - val_accuracy

In [50]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

In [55]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [58]:
#reverse_input_char_index
#reverse_target_char_index

In [59]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [61]:
for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

1/1 [==============================] - 0s 21ms/step
-
Input sentence: Go.
Decoded sentence: Marche.

1/1 [==============================] - 0s 24ms/step
-
Input sentence: Go.
Decoded sentence: Marche.

1/1 [==============================] - 0s 24ms/step
-
Input sentence: Go.
Decoded sentence: Marche.

1/1 [==============================] - 0s 24ms/step
-
Input sentence: Go.
Decoded sentence: Marche.

1/1 [==============================] - 0s 20ms/step
-
Input sentence: Hi.
Decoded sentence: Salut.

1/1 [==============================] - 0s 21ms/step
-
Input sentence: Hi.
Decoded sentence: Salut.

1/1 [==============================] - 0s 23ms/step
-
Input sentence: Run!
Decoded sentence: Fuyez !

1/1 [==============================] - 0s 21ms/step
-
Input sentence: Run!
Decoded sentence: Fuyez !

1/1 [==============================] - 0s 24ms/step
-
Input sentence: Run!
Decoded sentence: Fuyez !

1/1 [==============================] - 0s 24ms/step
-
Input sentence: Run!
Decoded sentenc